In [1]:
import pandas as pd
 
train_set = pd.read_table('train.tsv')
train_set.columns = ['id','label','statement','subjects','speaker','speakers job title','state info','party aff','barely true counts','false counts','half true counts','mostly true counts','pants on fire counts','venue']
train_set.head()

,id,label,statement,subjects,speaker,speakers job title,state info,party aff,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [2]:
train_set = train_set.drop(['id','subjects','speaker','speakers job title','state info','party aff','barely true counts','false counts','half true counts','mostly true counts','pants on fire counts','venue'], axis=1)
train_set.head()

,label,statement
0,half-true,When did the decline of coal start? It started...
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo..."
2,false,Health care reform legislation is likely to ma...
3,half-true,The economic turnaround started at the end of ...
4,true,The Chicago Bears have had more starting quart...


In [3]:
if(train_set['label'][0]=='half-true'):
    print('fvfv')

fvfv


In [4]:
for i in range(len(train_set)):
    if (train_set['label'][i] == 'mostly-true'):
        train_set['label'][i]='True'
    elif(train_set['label'][i] == 'true'):
        train_set['label'][i]='True'
    elif(train_set['label'][i] == 'half-true'):
        train_set['label'][i]='True'
    else :
        train_set['label'][i]='False'

train_set.head()
    

,label,statement
0,True,When did the decline of coal start? It started...
1,True,"Hillary Clinton agrees with John McCain ""by vo..."
2,False,Health care reform legislation is likely to ma...
3,True,The economic turnaround started at the end of ...
4,True,The Chicago Bears have had more starting quart...


In [5]:
print(train_set['label'].value_counts())
print(train_set.shape)
train_set.isnull().sum()

True     5752
False    4487
Name: label, dtype: int64
(10239, 2)


label        0
statement    0
dtype: int64

In [6]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stpwrds = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
for x in range(len(train_set)) :
    corpus = []
    review = train_set['statement'][x]
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    review = review.lower()
    review = nltk.word_tokenize(review)
    for y in review :
        if y not in stpwrds :
            corpus.append(lemmatizer.lemmatize(y))
    review = ' '.join(corpus)
    train_set['statement'][x] = review 

In [7]:
train_set.head()

,label,statement
0,True,decline coal start started natural gas took st...
1,True,hillary clinton agrees john mccain voting give...
2,False,health care reform legislation likely mandate ...
3,True,economic turnaround started end term
4,True,chicago bear starting quarterback last year to...


In [8]:
train_set['statement'][0]

'decline coal start started natural gas took started begin president george w bush administration'

# Feature Extraction 

# tfidf vectorizer

In [9]:
df1 = train_set.copy(deep=True)

In [10]:
x = df1.iloc[:,1]
y = df1.iloc[:,0]
y.head()

0     True
1     True
2    False
3     True
4     True
Name: label, dtype: object

In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20)

In [12]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
y_train.head()

(8191,)
(2048,)
(8191,)
(2048,)


865     False
5129    False
9087     True
9357    False
9147     True
Name: label, dtype: object

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, max_features = 200000, ngram_range = ( 1, 4 ),sublinear_tf = True )

vectorizer = vectorizer.fit(x_train)
tfidf_x_train = vectorizer.transform(x_train)
tfidf_x_test = vectorizer.transform(x_test)
tfidf_x_train.shape

(8191, 14948)

In [14]:
from sklearn.linear_model import LogisticRegression

m1 = LogisticRegression()
m1.fit(tfidf_x_train,y_train)

ypred_m1 = m1.predict(tfidf_x_test)
d1 = pd.DataFrame({'y_test':y_test,'ypred_m1':ypred_m1})
d1.head()

,y_test,ypred_m1
4692,True,True
6370,False,True
9903,False,False
7117,False,True
7028,False,False


In [15]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

print('\nClassification Report :-\n',classification_report(y_test,ypred_m1))
print('\nAccuracy Score : ',accuracy_score(y_test,ypred_m1))


Classification Report :-
               precision    recall  f1-score   support

       False       0.58      0.42      0.48       877
        True       0.64      0.77      0.70      1171

    accuracy                           0.62      2048
   macro avg       0.61      0.59      0.59      2048
weighted avg       0.61      0.62      0.61      2048


Accuracy Score :  0.619140625


In [16]:
from sklearn.svm import SVC

m2 = SVC(kernel='linear',max_iter=5000,C=1)
m2.fit(tfidf_x_train,y_train)

ypred_m2 = m2.predict(tfidf_x_test)
d2 = pd.DataFrame({'y_test':y_test,'ypred_m2':ypred_m2})
d2.head()

C:\Users\sanja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


,y_test,ypred_m2
4692,True,True
6370,False,True
9903,False,False
7117,False,True
7028,False,False


In [17]:
print('\nClassification Report :-\n',classification_report(y_test,ypred_m2))
print('\nAccuracy Score : ',accuracy_score(y_test,ypred_m2))


Classification Report :-
               precision    recall  f1-score   support

       False       0.56      0.44      0.49       877
        True       0.64      0.74      0.68      1171

    accuracy                           0.61      2048
   macro avg       0.60      0.59      0.59      2048
weighted avg       0.60      0.61      0.60      2048


Accuracy Score :  0.61083984375


# CountVectorizer 

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect = count_vect.fit(x_train)
cv_x_train = count_vect.transform(x_train)
cv_x_test = count_vect.transform(x_test)
cv_x_train.shape

(8191, 9710)

In [19]:
from sklearn.linear_model import LogisticRegression

m1 = LogisticRegression()
m1.fit(cv_x_train,y_train)

ypred_m1 = m1.predict(cv_x_test)
d1 = pd.DataFrame({'y_test':y_test,'ypred_m1':ypred_m1})
d1.head()

,y_test,ypred_m1
4692,True,True
6370,False,True
9903,False,False
7117,False,True
7028,False,False


In [20]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

print('\nClassification Report :-\n',classification_report(y_test,ypred_m1))
print('\nAccuracy Score : ',accuracy_score(y_test,ypred_m1))


Classification Report :-
               precision    recall  f1-score   support

       False       0.52      0.45      0.48       877
        True       0.62      0.69      0.66      1171

    accuracy                           0.59      2048
   macro avg       0.57      0.57      0.57      2048
weighted avg       0.58      0.59      0.58      2048


Accuracy Score :  0.58544921875


In [21]:
from sklearn.svm import SVC

m2 = SVC(kernel='linear',max_iter=5000,C=1)
m2.fit(cv_x_train,y_train)

ypred_m2 = m2.predict(cv_x_test)
d2 = pd.DataFrame({'y_test':y_test,'ypred_m2':ypred_m2})
d2.head()

C:\Users\sanja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


,y_test,ypred_m2
4692,True,False
6370,False,False
9903,False,True
7117,False,True
7028,False,True


In [22]:
print('\nClassification Report :-\n',classification_report(y_test,ypred_m2))
print('\nAccuracy Score : ',accuracy_score(y_test,ypred_m2))


Classification Report :-
               precision    recall  f1-score   support

       False       0.47      0.42      0.45       877
        True       0.60      0.64      0.62      1171

    accuracy                           0.55      2048
   macro avg       0.53      0.53      0.53      2048
weighted avg       0.54      0.55      0.55      2048


Accuracy Score :  0.548828125


# Doc2Vec

In [23]:
print(train_set.shape)
train_set.head()

(10239, 2)


,label,statement
0,True,decline coal start started natural gas took st...
1,True,hillary clinton agrees john mccain voting give...
2,False,health care reform legislation likely mandate ...
3,True,economic turnaround started end term
4,True,chicago bear starting quarterback last year to...


In [24]:
s = train_set['statement'][0]
print(s)
s.split()

decline coal start started natural gas took started begin president george w bush administration


['decline',
 'coal',
 'start',
 'started',
 'natural',
 'gas',
 'took',
 'started',
 'begin',
 'president',
 'george',
 'w',
 'bush',
 'administration']

In [25]:
import gensim
from gensim.models.doc2vec import TaggedDocument

docs = []

for i in range(10239):
    s = train_set['statement'][i]
    docs.append(TaggedDocument(words = s.split(),tags = [i]))
    

In [26]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=40, window=5,min_count=2, epochs=30)
model.build_vocab(docs)

In [27]:
model.train(docs, total_examples=model.corpus_count, epochs=model.epochs)

In [28]:
print(model.infer_vector(['decline coal start started natural gas took started begin president george w bush administration']))

[ 0.00176739  0.00814948 -0.00737249  0.00394914 -0.00222553  0.00524633
 -0.00062426 -0.01112326  0.00275794 -0.00434505  0.00603721  0.0002007
 -0.01018972  0.008202    0.00681654 -0.00633282  0.00241731 -0.01007165
  0.00799781 -0.00426448  0.00474346 -0.01014731 -0.00821144  0.00316921
  0.00812131 -0.00308478  0.01027264 -0.00194223 -0.00889201 -0.00894945
 -0.0057515  -0.00154386  0.01018333  0.01070065  0.01003412 -0.00709868
 -0.00028009 -0.00816162  0.00587921  0.00928052]


In [29]:
dv = pd.DataFrame()
for i in range(len(x)):
    d = pd.DataFrame([model.infer_vector(x[i].split())])
    dv = dv.append(d)

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

...

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dv = dv.append(d)
C:\Users\sanja\AppData\Local\Temp\ipykernel_26576\1613288449.py:4: FutureWarning: The frame.append method is deprecated 

In [30]:
print(dv.shape)

(10239, 40)


In [31]:
dv_x_train,dv_x_test,y_train,y_test = train_test_split(dv,y,test_size=0.20)

In [32]:
from sklearn.linear_model import LogisticRegression

m1 = LogisticRegression()
m1.fit(dv_x_train,y_train)

ypred_m1 = m1.predict(dv_x_test)
d1 = pd.DataFrame({'y_test':y_test,'ypred_m1':ypred_m1})
d1.head()

,y_test,ypred_m1
1348,True,True
6291,True,True
9697,True,True
10089,True,True
249,True,True


In [ ]:
for i in ypred_m1:
    if

In [33]:
print('\nClassification Report :-\n',classification_report(y_test,ypred_m1))
print('\nAccuracy Score : ',accuracy_score(y_test,ypred_m1))


Classification Report :-
               precision    recall  f1-score   support

       False       0.55      0.29      0.38       889
        True       0.60      0.82      0.69      1159

    accuracy                           0.59      2048
   macro avg       0.58      0.55      0.54      2048
weighted avg       0.58      0.59      0.56      2048


Accuracy Score :  0.58935546875


In [34]:
from sklearn.svm import SVC

m2 = SVC(kernel='linear',max_iter=5000,C=1)
m2.fit(dv_x_train,y_train)

ypred_m2 = m2.predict(dv_x_test)
d2 = pd.DataFrame({'y_test':y_test,'ypred_m2':ypred_m2})
d2.head()

C:\Users\sanja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


,y_test,ypred_m2
1348,True,True
6291,True,True
9697,True,True
10089,True,True
249,True,True


In [35]:
print('\nClassification Report :-\n',classification_report(y_test,ypred_m2))
print('\nAccuracy Score : ',accuracy_score(y_test,ypred_m2))


Classification Report :-
               precision    recall  f1-score   support

       False       0.56      0.14      0.22       889
        True       0.58      0.92      0.71      1159

    accuracy                           0.58      2048
   macro avg       0.57      0.53      0.46      2048
weighted avg       0.57      0.58      0.50      2048


Accuracy Score :  0.578125


# Waste trial

In [36]:
model1 = gensim.models.doc2vec.Doc2Vec(vector_size=1, window=5,min_count=2, epochs=30)
model1.build_vocab(docs)
model1.train(docs, total_examples=model1.corpus_count, epochs=model1.epochs)

In [37]:
x1_train = df1.iloc[:8191,1]
y1_train = df1.iloc[:8191,0]
x1_test = df1.iloc[8191:,1]
y1_test = df1.iloc[8191:,0]
print(x1_train.shape)
x1_train.head()

(8191,)


0    decline coal start started natural gas took st...
1    hillary clinton agrees john mccain voting give...
2    health care reform legislation likely mandate ...
3                 economic turnaround started end term
4    chicago bear starting quarterback last year to...
Name: statement, dtype: object

In [38]:
dv1 = [model1.infer_vector([x1_train[0]]),]
for i in range(1,8191):
    if(y1_train[i]=='True'):
        arr = model1.infer_vector([x1_train[i]])
        dv1 = np.append(dv1, [arr], axis=0)

NameError: name 'np' is not defined

In [ ]:
print(dv1)
print(dv1.shape)

In [ ]:
print(np.mean(dv1, axis=1))

In [ ]:
import numpy as np
a = np.array([[1, 2], [3, 4],[5,6]])

In [ ]:
from scipy.spatial import distance
def cosine_distance_countvectorizer_method(s1, s2):
    # distance of similarity
    cosine = distance.cosine(s1,s1)
    print(c)
    print('Similarity of two dataset are equal to ',round((1-cosine)*100,2),'%')
    return cosine

In [ ]:
ypred_d = np.empty((len(y1_test),1),dtype=str)
for i in range(8191,10239):
    v = model1.infer_vector([x1_test[i]])
    c = cosine_distance_countvectorizer_method(v,dv1)
    if(c>.5):
        np.append(ypred_d, 'True')
    else:
        np.append(ypred_d, 'False')

In [ ]:
print(ypred_d)

In [ ]:
ypred_m2[0]